In [106]:
import warnings
warnings.filterwarnings(action="ignore")
import os
import ast
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.patches as mpatches
import seaborn as sn
from tqdm.std import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math as math
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
class Config:
    data_dir = '../data/'
    working_dir = '../src/'
    models_dir = '../models/'
    submissions_dir = '../submissions'

In [3]:
#importing the data
train = pd.read_csv("../input/assgn3/Train.csv")
metadata = pd.read_csv("../input/assgn3/metadata.csv")
test = pd.read_csv("../input/assgn3/Test.csv")
ss = pd.read_csv("../input/assgn3/Test.csv")

In [4]:
print(metadata.isna().sum())#checking the na values

ID                         0
RegistrationDate           0
Deposit                    0
UpsellDate             36370
AccessoryRate              0
PaymentMethod              0
rateTypeEntity             0
RatePerUnit                0
DaysOnDeposit              0
MainApplicantGender        0
Age                     6939
Region                  1934
Town                       0
Occupation                 0
SupplierName               0
Term                       0
TotalContractValue         0
ExpectedTermDate           0
FirstPaymentDate           0
LastPaymentDate            0
dtype: int64


In [5]:
metadata[metadata.isna().Region].head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
17,ID_AYIBQUP,2015-12-12 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,44.0,NaN,UNKNOWN,Other,d_light,364,14740.0,2016-12-10 00:00:00,2015-12-12 07:00:38,2020-09-02 20:30:53
51,ID_35MNQL1,2015-12-06 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,42.0,NaN,UNKNOWN,Farmer,d_light,364,14740.0,2016-12-04 00:00:00,2015-12-07 06:21:45,2017-11-18 11:07:25
84,ID_RWA0Q3U,2017-07-12 14:44:48,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,52.0,NaN,UNKNOWN,Government Employee,d_light,364,16560.0,2018-07-11 14:44:48,2017-07-12 14:45:19,2020-09-02 20:31:12
85,ID_QWI7WWN,2015-11-24 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,40.0,NaN,UNKNOWN,Other,d_light,364,14740.0,2016-11-22 00:00:00,2015-11-24 14:02:51,2016-11-25 14:58:45


In [6]:
metadata.Region.value_counts()

Nyanza                6998
North Rift            5344
Nairobi Region        5056
South Rift            4759
Coast Region          4671
Western               4508
Mount Kenya Region    4073
Name: Region, dtype: int64

In [8]:
pd.merge(left=metadata[metadata.isna().Region], right=test, on="ID").head(2)
#test data also contains empty values in region therefore we need to treat as a category.

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_UOT1MF3,2017-07-13 08:56:05,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,54.0,NaN,UNKNOWN,Business,d_light,364,16560.0,2018-07-12 08:56:05,2017-07-13 08:56:23,2020-09-02 20:31:28,"['07-2017', '08-2017', '09-2017', '10-2017', '11-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '10-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '07-2019']","[2720.0, 1080.0, 1200.0, 1000.0, 960.0, 1160.0, 1120.0, 600.0, 1040.0, 840.0, 680.0, 640.0, 680.0, 520.0, 80.0, 80.0, 40.0, 40.0, 240.0, 40.0, 80.0, 40.0]"
1,ID_290EB8N,2017-07-05 09:06:30,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,31.0,NaN,UNKNOWN,Labourer,d_light,364,16560.0,2018-07-04 09:06:30,2017-07-05 08:54:37,2020-09-02 20:35:26,"['07-2017', '08-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018']","[2710.0, 120.0, 200.0, 300.0, 50.0, 250.0, 550.0, 100.0, 500.0, 350.0, 350.0, 950.0, 200.0]"


In [9]:
metadata.Region.fillna(value="Other", inplace=True)#replacing na with other in regions

In [10]:
metadata.UpsellDate.fillna(value=0, inplace=True)#replacing NA with 0 in UpSellDate

In [11]:
pd.merge(left=metadata[metadata.isna().Age], right=test, on="ID").head(2)
##now just using mean here to remove 0 in metadata.


,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_VJ80SX2,2015-12-14 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,NaN,Mount Kenya Region,Embu,Business,d_light,364,14740.0,2016-12-12 00:00:00,2015-12-14 14:20:26,2016-12-12 11:59:20,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016']","[3000.0, 850.0, 750.0, 1500.0, 650.0, 1250.0, 1000.0]"
1,ID_ZLW8XIB,2015-12-05 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,NaN,Mount Kenya Region,Embu,Farmer,d_light,364,14740.0,2016-12-03 00:00:00,2015-12-05 15:28:45,2017-04-01 19:19:19,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016']","[3200.0, 735.0, 1320.0, 1000.0, 1170.0, 866.0, 730.0, 100.0, 100.0, 450.0, 700.0]"


In [12]:
metadata.Age.fillna(value=round(metadata.Age.mean()), inplace=True)#replacing the NA in age with mean value of age

In [13]:
merged = pd.merge(left=metadata, right=pd.concat(objs=[train, test]).fillna(value=0), on="ID")
merged.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_K00S4N4,2015-12-10 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']","[3050.0, 1050.0, 910.0, 1050.0, 1050.0]",1225.0,1050.0,1190.0,525.0,1750.0,1890.0
1,ID_6L67PAA,2015-12-09 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18,"['12-2015', '01-2016', '02-2016', '03-2016', '05-2016', '07-2016']","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]",0.0,0.0,0.0,0.0,0.0,0.0
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016']","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 980.0, 915.0, 735.0]",980.0,980.0,1225.0,980.0,935.0,355.0
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,43.0,Other,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54,"['11-2015', '12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '10-2016']","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 980.0, 1225.0, 735.0, 490.0, 250.0]",250.0,1000.0,250.0,500.0,560.0,1150.0
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016', '11-2016']","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, 280.0, 665.0, 770.0, 420.0, 525.0, 735.0]",630.0,805.0,700.0,855.0,245.0,775.0


In [14]:
merged.drop(["FirstPaymentDate"],axis=1,inplace=True)

In [15]:
#defining a function to calculate time diff from dates
def diff(a,b):
    start=b.split('-')
    end=a.split('-')
    return int(end[1])-int(start[1])+12*(int(end[0])-int(start[0]))

In [16]:
#merged['month']=merged['RegistrationDate']
#merged['year']=merged['RegistrationDate']
merged['time']=merged['RegistrationDate']
merged['PaidAmount']=merged['TotalContractValue']
merged['PeriodDiff']=merged['TotalContractValue']
for i in tqdm(range(0,merged.shape[0]),desc='Row'):
    year=int(merged.RegistrationDate.iloc[i].split('-')[0])
    month=int(merged.RegistrationDate.iloc[i].split('-')[1])
    time=np.ceil(merged.loc[i,'Term']/30)
    merged.loc[i,'time']=time
    if merged.UpsellDate.iloc[i]!=0:
        merged.loc[i,'UpsellDate']=diff(merged.UpsellDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'LastPaymentDate']=diff(merged.LastPaymentDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'ExpectedTermDate']=diff(merged.ExpectedTermDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'m1']=merged.loc[i,'m1']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m2']=merged.loc[i,'m2']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m3']=merged.loc[i,'m3']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m4']=merged.loc[i,'m4']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m5']=merged.loc[i,'m5']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m6']=merged.loc[i,'m6']/merged.loc[i,'TotalContractValue']
    dates=[]
    for j in merged.loc[i,'TransactionDates'].split("'"):
        if '-' in j:
            date=j.split('-')
            dates.append((int(date[0])-month+12*(int(date[1])-year))/time)
    merged.at[i,'TransactionDates']=dates
    history=[]
    for j in merged.loc[i,'PaymentsHistory'].split("[")[1].split("]")[0].split(","):
        if '.' in j:
            history.append(float(j)/merged.loc[i,'TotalContractValue'])
    merged.at[i,'PaymentsHistory']=history
    merged.loc[i,'PaidAmount']=sum(history)
    merged.loc[i,'PeriodDiff']=merged.loc[i,'LastPaymentDate']-dates[-1]
merged.drop(["RegistrationDate","Term"],axis=1,inplace=True)

Row: 100%|██████████| 37343/37343 [04:40<00:00, 133.20it/s]


In [17]:
merged.tail(2)

,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
37341,ID_GHHAQ9D,2400,0,0.0,FINANCED,DAILY,50,3,Male,27.0,Other,UNKNOWN,Labourer,d_light,14400.0,1.0,1.75,"[0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]","[0.18194444444444444, 0.09027777777777778, 0.08333333333333333, 0.07291666666666667, 0.0763888888888889, 0.04513888888888889, 0.010416666666666666, 0.0038194444444444443, 0.021875]",0.034722,0.059028,0.045139,0.048611,0.041667,0.048611,8.0,0.586111,0.75
37342,ID_PAI1FJK,2400,0,0.0,FINANCED,DAILY,50,3,Male,36.0,Coast Region,Mombasa,Farmer,d_light,14400.0,1.0,1.875,"[0.0, 0.125, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125]","[0.1701388888888889, 0.003472222222222222, 0.003472222222222222, 0.09375, 0.03125, 0.07291666666666667, 0.04861111111111111, 0.0798611111111111, 0.05277777777777778]",0.100694,0.055556,0.017361,0.003472,0.010417,0.045139,8.0,0.556250,0.75


In [18]:
#label encoding the required data
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
category_list=['PaymentMethod','rateTypeEntity','MainApplicantGender','Region','Town','Occupation','SupplierName']
for s in category_list:
    merged[s] = lb_make.fit_transform(merged[s])
    print(lb_make.transform(lb_make.classes_))
    print(lb_make.classes_)
merged.head(2)

[0]
['FINANCED']
[0 1 2]
['DAILY' 'MONTHLY' 'WEEKLY']
[0 1]
['Female' 'Male']
[0 1 2 3 4 5 6 7]
['Coast Region' 'Mount Kenya Region' 'Nairobi Region' 'North Rift'
 'Nyanza' 'Other' 'South Rift' 'Western']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
['Baringo' 'Bomet' 'Bungoma' 'Busia' 'Elgeyo/Marakwet' 'Embu' 'Garissa'
 'Homa Bay' 'Isiolo' 'Kajiado' 'Kakamega' 'Kericho' 'Kiambu' 'Kilifi'
 'Kirinyaga' 'Kisii' 'Kisumu' 'Kitui' 'Kwale' 'Laikipia' 'Lamu' 'Machakos'
 'Makueni' 'Mandera' 'Marsabit' 'Meru' 'Migori' 'Mombasa' 'Muranga'
 'Nairobi City' 'Nakuru' 'Nandi' 'Narok' 'Nyamira' 'Nyandarua' 'Nyeri'
 'Samburu' 'Siaya' 'Taita/Taveta' 'Tana River' 'Tharaka-Nithi'
 'Trans Nzoia' 'Turkana' 'UNKNOWN' 'Uasin Gishu' 'Vihiga' 'Wajir'
 'West Pokot']
[0 1 2 3 4 5 6]
['Business' 'Driver/Motorbike Rider' 'Farmer' 'Government Employee'
 'Labourer' 'Other' 'Teacher']
[0]
['d_light']


,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,0,0,35,7,1,41.0,1,5,5,0,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,0,0,35,7,1,33.0,0,13,5,0,14740.0,0.923077,4.076923,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462


In [19]:
## As these two variable has only one class.
merged.drop(['PaymentMethod','SupplierName'],axis=1,inplace=True)

In [20]:
merged.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,0,35,7,1,41.0,1,5,5,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,0,35,7,1,33.0,0,13,5,14740.0,0.923077,4.076923,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462
2,ID_102CV85,2000,1.928571,35.0,0,35,7,0,48.0,2,22,0,29480.0,0.928571,1.0,"[0.0, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.2857142857142857, 0.35714285714285715, 0.42857142857142855, 0.5, 0.5714285714285714]","[0.14399592944369063, 0.03324287652645862, 0.024932157394843963, 0.024932157394843963, 0.049864314789687926, 0.024932157394843963, 0.03324287652645862, 0.031037991858887382, 0.024932157394843963]",0.033243,0.033243,0.041554,0.033243,0.031716,0.012042,14.0,0.391113,0.428571
3,ID_HXBJFHB,2000,0,0.0,0,35,7,0,43.0,5,43,6,14740.0,0.923077,1.384615,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.8461538461538461]","[0.1523066485753053, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.06648575305291723, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.049864314789687926, 0.03324287652645862, 0.016960651289009497]",0.016961,0.067843,0.016961,0.033921,0.037992,0.078019,13.0,0.751018,0.538462
4,ID_3K9VZ5J,2000,0,0.0,0,35,7,0,56.0,1,14,5,14740.0,0.846154,1.307692,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.7692307692307693, 0.8461538461538461]","[0.1865671641791045, 0.06784260515603799, 0.05088195386702849, 0.06784260515603799, 0.06478968792401628, 0.05970149253731343, 0.018995929443690638, 0.04511533242876527, 0.05223880597014925, 0.028493894165535955, 0.03561736770691994, 0.049864314789687926]",0.042741,0.054613,0.047490,0.058005,0.016621,0.052578,13.0,0.727951,0.461538


In [21]:
merged.to_csv("Final.csv",index=False)#storing the preprocessed data

##Advance:: Can use clustering/estimation to get age.
##Advance:: Care for Days on deposit(Some finance has to be read)
##Advance:: Care for deposit in paid amount.

In [22]:
#using tsfresh lib to extract features from the time series data
from tsfresh import extract_features
df_final=pd.DataFrame(columns=["time","money","id"])
for k in tqdm(range(0,merged.shape[0])):
    df_temp=pd.DataFrame({"time":merged.loc[k,"TransactionDates"],"money":merged.loc[k,"PaymentsHistory"]})
    df_temp['id']=merged.loc[k,"ID"]
    df_final=pd.concat([df_final, df_temp], ignore_index=True)
print(df_final.shape)

100%|██████████| 37343/37343 [02:48<00:00, 221.57it/s]

(607851, 3)


In [31]:
#extract_features=extract_features(df_final,column_id='id',column_sort="time").dropna(axis=1)
extract_features=pd.read_csv('../input/features/Features') #extracting the features from time series

In [32]:
extract_features.head(5)

,Unnamed: 0,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","mone

In [242]:
#creating a final feature set with both static and time varying features
final_data=merged.iloc[:,0:15]
final_data.index=final_data.iloc[:,1]
final_data.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate
Deposit,,,,,,,,,,,,,,,
2000,ID_K00S4N4,2000,0,0.0,0,35,7,1,41.0,1,5,5,14740.0,0.923077,0.769231
2000,ID_6L67PAA,2000,0,0.0,0,35,7,1,33.0,0,13,5,14740.0,0.923077,4.076923
2000,ID_102CV85,2000,1.928571,35.0,0,35,7,0,48.0,2,22,0,29480.0,0.928571,1.0
2000,ID_HXBJFHB,2000,0,0.0,0,35,7,0,43.0,5,43,6,14740.0,0.923077,1.384615
2000,ID_3K9VZ5J,2000,0,0.0,0,35,7,0,56.0,1,14,5,14740.0,0.846154,1.307692


In [26]:
extract_features.to_csv("Features")# saving the features data

In [243]:
final_data=pd.merge(left=final_data,right=extract_features,on='ID')#mergeing the static and time varying features



In [60]:
final_data.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,Unnamed: 0_x,money__variance_larger_than_standard_deviation_x,money__has_duplicate_max_x,money__has_duplicate_min_x,money__has_duplicate_x,money__sum_values_x,money__abs_energy_x,money__mean_abs_change_x,money__mean_change_x,money__mean_second_derivative_central_x,money__median_x,money__mean_x,money__length_x,money__standard_deviation_x,money__variation_coefficient_x,money__variance_x,money__skewness_x,money__kurtosis_x,money__root_mean_square_x,money__absolute_sum_of_changes_x,money__longest_strike_below_mean_x,money__longest_strike_above_mean_x,money__count_above_mean_x,money__count_below_mean_x,money__last_location_of_maximum_x,money__first_location_of_maximum_x,money__last_location_of_minimum_x,money__first_location_of_minimum_x,money__percentage_of_reoccurring_values_to_all_values_x,money__percentage_of_reoccurring_datapoints_to_all_datapoints_x,money__sum_of_reoccurring_values_x,money__sum_of_reoccurring_data_points_x,money__ratio_value_number_to_time_series_length_x,money__maximum_x,money__minimum_x,money__time_reversal_asymmetry_statistic__lag_1_x,money__time_reversal_asymmetry_statistic__lag_2_x,money__time_reversal_asymmetry_statistic__lag_3_x,money__c3__lag_1_x,money__c3__lag_2_x,money__c3__lag_3_x,money__cid_ce__normalize_True_x,money__cid_ce__normalize_False_x,money__symmetry_looking__r_0.0_x,money__symmetry_looking__r_0.05_x,money__symmetry_looking__r_0.1_x,money__symmetry_looking__r_0.15000000000000002_x,money__symmetry_looking__r_0.2_x,money__symmetry_looking__r_0.25_x,money__symmetry_looking__r_0.30000000000000004_x,money__symmetry_looking__r_0.35000000000000003_x,money__symmetry_looking__r_0.4_x,money__symmetry_looking__r_0.45_x,money__symmetry_looking__r_0.5_x,money__symmetry_looking__r_0.55_x,money__symmetry_looking__r_0.6000000000000001_x,money__symmetry_looking__r_0.65_x,money__symmetry_looking__r_0.7000000000000001_x,money__symmetry_looking__r_0.75_x,money__symmetry_looking__r_0.8_x,money__symmetry_looking__r_0.8500000000000001_x,money__symmetry_looking__r_0.9_x,money__symmetry_looking__r_0.9500000000000001_x,money__large_standard_deviation__r_0.05_x,money__large_standard_deviation__r_0.1_x,money__large_standard_deviation__r_0.15000000000000002_x,money__large_standard_deviation__r_0.2_x,money__large_standard_deviation__r_0.25_x,money__large_standard_deviation__r_0.30000000000000004_x,money__large_standard_deviation__r_0.35000000000000003_x,money__large_standard_deviation__r_0.4_x,money__large_standard_deviation__r_0.45_x,money__large_standard_deviation__r_0.5_x,money__large_standard_deviation__r_0.55_x,money__large_standard_deviation__r_0.6000000000000001_x,money__large_standard_deviation__r_0.65_x,money__large_standard_deviation__r_0.7000000000000001_x,money__large_standard_deviation__r_0.75_x,money__large_standard_deviation__r_0.8_x,money__large_standard_deviation__r_0.8500000000000001_x,money__large_standard_deviation__r_0.9_x,money__large_standard_deviation__r_0.9500000000000001_x,money__quantile__q_0.1_x,money__quantile__q_0.2_x,money__quantile__q_0.3_x,money__quantile__q_0.4_x,money__quantile__q_0.6_x,money__quantile__q_0.7_x,money__quantile__q_0.8_x,money__quantile__q_0.9_x,money__autocorrelation__lag_0_x,money__autocorrelation__lag_1_x,money__autocorrelation__lag_2_x,money__autocorrelation__lag_3_x,money__autocorrelation__lag_4_x,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40_x","money__agg_autocorrelation__f_agg_""median""__maxlag_40_x","money__agg_autocorrelation__f_agg_""var""__maxlag_40_x",money__partial_autocorrelation__lag_0_x,money__partial_autocorrelation__lag_1_x,money__number_cwt_peaks__n_1_x,money__number_cwt_peaks__n_5_x,money__number_peaks__n_1_x,money__number_peaks__n_3_x,money__number_peaks__n_5_x,money__number_peaks__n_10_x,money__number_peaks__n_50_x,money__binned_entropy__max_bins_10_x,money__inde

# x=final_features
y=merged.iloc[:,18]


In [244]:
#Changing the object type data to float type
final_data["ExpectedTermDate"]=final_data.ExpectedTermDate.astype(float)
final_data["LastPaymentDate"]=final_data.LastPaymentDate.astype(float)
#final_data['ID']=final_data.ID.astype(float)
final_data['UpsellDate']=final_data.UpsellDate.astype(float)
#print(list(final_data.columns.values))
final_data.dtypes

ID                                                                     object
Deposit                                                                 int64
UpsellDate                                                            float64
AccessoryRate                                                         float64
rateTypeEntity                                                          int64
RatePerUnit                                                             int64
DaysOnDeposit                                                           int64
MainApplicantGender                                                     int64
Age                                                                   float64
Region                                                                  int64
Town                                                                    int64
Occupation                                                              int64
TotalContractValue                                              

In [246]:
y=merged.loc[:,'m1']

x=final_data
x=x.drop(columns=['ID','Unnamed: 0','money__value_count__value_1','money__number_crossing_m__m_1'])#dropping unwanted features

import re
x = x.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

#x=x.drop_duplicates()
#y=y.drop_duplicates()
#x.reset_index(inplace=True)


In [247]:
#implementing the lightgbm model
from lightgbm import LGBMRegressor
lgbm=LGBMRegressor()
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 


#x_train=x_train.drop(columns=['ExpectedTermDate','LastPaymentDate'])
#x["LastPaymentDate"] = x.LastPaymentDate.astype(float)
#x["ExpectedTermDate"]=x.ExpectedTermDate.astype(float)
#x_train.astype({'LastPaymentDate':'float64'}).dtypes

#x_train.head(2)

x_train=x_train.drop_duplicates
lgbm.fit(x_train,y_train)


In [248]:

lgbm.fit(x_train,y_train)

LGBMRegressor()

In [249]:
#using lgbm for m1
pred_m1=lgbm.predict(x_test)
MSE=[]
mse_m1=math.sqrt(mean_squared_error(y_test,pred_m1))
print(mse_m1)
MSE.append(mse_m1)

0.025446517901482783


In [250]:
y=merged.loc[:,'m2']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 

In [251]:
#using lgbm for m2
pred_m2=lgbm.predict(x_test)
mse_m2=math.sqrt(mean_squared_error(y_test,pred_m2))
print(mse_m2)
MSE.append(mse_m2)

0.025553753928544833


In [252]:
y=merged.loc[:,'m3']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 

In [253]:
#using lgbm for m3
pred_m3=lgbm.predict(x_test)
mse_m3=math.sqrt(mean_squared_error(y_test,pred_m3))
print(mse_m3)
MSE.append(mse_m3)

0.029640988262330737


In [254]:
y=merged.loc[:,'m4']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 

In [255]:
#using lgbm for m4
pred_m4=lgbm.predict(x_test)
mse_m4=math.sqrt(mean_squared_error(y_test,pred_m4))
print(mse_m4)
MSE.append(mse_m4)

0.029999491458203645


In [256]:
y=merged.loc[:,'m5']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 

In [257]:
#using lgbm for m5
pred_m5=lgbm.predict(x_test)
mse_m5=math.sqrt(mean_squared_error(y_test,pred_m5))
print(mse_m5)
MSE.append(mse_m5)

0.027914915852809544


In [258]:
y=merged.loc[:,'m6']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.1, random_state=0) 

In [259]:
#using lgbm for m6
pred_m6=lgbm.predict(x_test)
mse_m6=math.sqrt(mean_squared_error(y_test,pred_m6))
print(mse_m6)
MSE.append(mse_m6)

0.042786732119347824


In [260]:

print(MSE)#MSE of simple split train and test data

[0.025446517901482783, 0.025553753928544833, 0.029640988262330737, 0.029999491458203645, 0.027914915852809544, 0.042786732119347824]


In [261]:
print(x.index)

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            37333, 37334, 37335, 37336, 37337, 37338, 37339, 37340, 37341,
            37342],
           dtype='int64', length=37343)


In [277]:
#using K-fold to train the model
from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state=None)
s=0
y=merged.loc[:,'m1']
MSE_kfold=[]
#print(x.shape)
for train_index, test_index in kf.split(x):
    #print(train_index,test_index)
    #print(train_index.shape)
    #print(test_index.shape)
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    #y_train,y_test=y[train_index],y[test_index]
    lgbm.fit(x_train,y_train)
    pred_m1=lgbm.predict(x_test)
    mse=math.sqrt(mean_squared_error(y_test,pred_m1))
    s=s+mse
mse=s/10
print(mse)
MSE_kfold.append(mse)

0.023044908968476562


In [278]:
y=merged.loc[:,'m2']
s=0
#print(x.shape)
for train_index, test_index in kf.split(x):
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    lgbm.fit(x_train,y_train)
    pred_m2=lgbm.predict(x_test)
    mse=math.sqrt(mean_squared_error(y_test,pred_m2))
    s=s+mse
mse=s/10
print(mse)
MSE_kfold.append(mse)

0.025262900842934966


In [279]:
y=merged.loc[:,'m3']
s=0
#print(x.shape)
for train_index, test_index in kf.split(x):
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    lgbm.fit(x_train,y_train)
    pred_m3=lgbm.predict(x_test)
    mse=math.sqrt(mean_squared_error(y_test,pred_m3))
    s=s+mse
mse=s/10
print(mse)
MSE_kfold.append(mse)

0.02413550580114242


In [280]:
y=merged.loc[:,'m4']
s=0
#print(x.shape)
for train_index, test_index in kf.split(x):
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    lgbm.fit(x_train,y_train)
    pred_m4=lgbm.predict(x_test)
    mse=math.sqrt(mean_squared_error(y_test,pred_m4))
    s=s+mse
mse=s/10
print(mse)
MSE_kfold.append(mse)

0.02490535110329225


In [281]:
y=merged.loc[:,'m5']
s=0

#print(x.shape)
for train_index, test_index in kf.split(x):
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    lgbm.fit(x_train,y_train)
    pred_m5=lgbm.predict(x_test)
    mse=math.sqrt(mean_squared_error(y_test,pred_m5))
    s=s+mse
mse=s/10
print(mse)
MSE_kfold.append(mse)

0.025624118927691914


In [305]:
y=merged.loc[:,'m6']
s=0
#print(x.shape)
for train_index, test_index in kf.split(x):
    x_train,x_test=x[train_index[0]:train_index[-1]],x[test_index[0]:test_index[-1]]
    y_train,y_test=y[train_index[0]:train_index[-1]],y[test_index[0]:test_index[-1]]
    lgbm.fit(x_train,y_train)
    pred_m6=lgbm.predict(x_test)
    
    mse=math.sqrt(mean_squared_error(y_test,pred_m6))
    s=s+mse
    mse=s/10
print(mse)
MSE_kfold.append(mse)

0.03256473058052439


In [283]:
print(MSE_kfold)#MSE for KFold 

[0.023044908968476562, 0.025262900842934966, 0.02413550580114242, 0.02490535110329225, 0.025624118927691914, 0.03256473058052439]


In [307]:

final_data.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,Unnamed: 0,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quant

In [308]:
test_id=test.loc[:,'ID']
test_id.head(5)

0    ID_6L67PAA
1    ID_VJ80SX2
2    ID_7OU9HLK
3    ID_WVWTPGK
4    ID_04DSDQS
Name: ID, dtype: object

In [312]:
final_data.index=final_data.loc[:,'ID']

final_data=final_data.drop(columns=['ID','Unnamed: 0','money__value_count__value_1','money__number_crossing_m__m_1'])

In [313]:
final_data.head(5)

,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,mon

In [317]:
ID=final_data.iloc[test_index[0]:test_index[-1],0]
#print(ID.size)
#print(pred_m1.size)
frame={'ID':ID,'m1':pred_m1,'m2':pred_m2,'m3':pred_m3,'m4':pred_m4,'m5':pred_m5,'m6':pred_m6}
submission=pd.DataFrame(frame)
submission.head(5)


,ID,m1,m2,m3,m4,m5,m6
ID,,,,,,,
ID_51P3TQH,2000,0.062430,0.046545,0.057302,0.059913,0.053332,0.029506
ID_SESYIX1,2000,0.009653,0.008514,0.008778,0.009871,0.008398,0.015357
ID_TDYY8AA,2000,0.043862,0.039183,0.026966,0.034509,0.031697,0.015253
ID_UHNVM6A,2000,0.026322,0.013039,0.024236,0.025120,0.019802,0.020980
ID_WUQX03C,2000,0.014582,0.014843,0.020983,0.019990,0.019895,0.022048


In [299]:
submission.to_csv("submission.csv",index=False)

I was not able to specifically take out the Values of m1 ,m2,m3... corresponding to test data ,due to which my submission was not accepted on zindi.Please see to it  .
Thank you